In [1]:
list=[['id_1','int_key','1'],
['id_1','string_key','asd'],
['id_1','double_key','null'],
['id_2','double_key','2.0']]

df=spark.createDataFrame(list, ['id','key','value'])
df.show()

+----+----------+-----+
 id| key|value|
+----+----------+-----+
id_1| int_key| 1|
id_1|string_key| asd|
id_1|double_key| null|
id_2|double_key| 2.0|
+----+----------+-----+

In [2]:
df.groupBy("id").pivot("key").agg(F.first("value")).select("id", F.col("double_key").cast("double"),F.col("int_key").cast("int"),F.col("string_key").cast("string")).show()

+----+----------+-------+----------+
 id|double_key|int_key|string_key|
+----+----------+-------+----------+
id_2| 2.0| null| null|
id_1| null| 1| asd|
+----+----------+-------+----------+

In [3]:
from pyspark.sql import functions as F
df.groupBy("id")\
  .agg(F.map_from_arrays(F.collect_list("key").alias("key"), F.collect_list("value").alias("value")).alias("mapped")).show(truncate=False)

+----+-----------------------------------------------------+
id |mapped |
+----+-----------------------------------------------------+
id_2|[double_key -> 2.0] |
id_1|[int_key -> 1, string_key -> asd, double_key -> null]|
+----+-----------------------------------------------------+

In [4]:
df.withColumn('concat',F.split((F.concat_ws(',',(*(x for x in df.columns[1:])))),',')).show()



'''.drop(*(x for x in df.columns[1:])).groupBy().pivot(df.columns[0]).agg(F.first("concat"))'''

+----+----------+-----+------------------+
 id| key|value| concat|
+----+----------+-----+------------------+
id_1| int_key| 1| [int_key, 1]|
id_1|string_key| asd| [string_key, asd]|
id_1|double_key| null|[double_key, null]|
id_2|double_key| 2.0| [double_key, 2.0]|
+----+----------+-----+------------------+

Out[50]: '.drop(*(x for x in df.columns[1:])).groupBy().pivot(df.columns[0]).agg(F.first("concat"))'

In [5]:
from pyspark.sql.types import *
schema=StructType([StructField("double_key", DoubleType(), True),
                  StructField("int_key", IntegerType(), True),
                  StructField("string_key", StringType(), True)])
df1.withColumn("mapped1", F.from_json("mapped",schema)).show(truncate=False)

--------------------------------------------------------------------------- 
 Py4JJavaError Traceback (most recent call last)
 /databricks/spark/python/pyspark/sql/utils.py in deco (*a, **kw) 
 62 try : 
 ---> 63 return f ( * a , ** kw ) 
 64 except py4j . protocol . Py4JJavaError as e : 

 /databricks/spark/python/lib/py4j-0.10.7-src.zip/py4j/protocol.py in get_return_value (answer, gateway_client, target_id, name) 
 327 "An error occurred while calling {0}{1}{2}.\n" . 
 --> 328 format(target_id, ".", name), value)
 329 else : 

 Py4JJavaError : An error occurred while calling o2128.withColumn.
: org.apache.spark.sql.AnalysisException: cannot resolve 'jsontostructs(`mapped`)' due to data type mismatch: argument 1 requires string type, however, '`mapped`' is of map<string,string> type.;;
'Project [id#16, mapped#883, jsontostructs(StructField(double_key,DoubleType,true), StructField(int_key,IntegerType,true), StructField(string_key,StringType,true), mapped#883, Some(Etc/UTC)) AS mapped1#927]
+- Aggregate [id#16], [id#16, map_from_arrays(collect_list(key#17, 0, 0), collect_list(value#18, 0, 0)) AS mapped#883]
 +- LogicalRDD [id#16, key#17, value#18], false

	at org.apache.spark.sql.catalyst.analysis.package$AnalysisErrorAt.failAnalysis(package.scala:42)
	at org.apache.spark.sql.catalyst.analysis.CheckAnalysis$$anonfun$checkAnalysis$1$$anonfun$apply$3.applyOrElse(CheckAnalysis.scala:126)
	at org.apache.spark.sql.catalyst.analysis.CheckAnalysis$$anonfun$checkAnalysis$1$$anonfun$apply$3.applyOrElse(CheckAnalysis.scala:111)
	at org.apache.spark.sql.catalyst.trees.TreeNode$$anonfun$6.apply(TreeNode.scala:303)
	at org.apache.spark.sql.catalyst.trees.TreeNode$$anonfun$6.apply(TreeNode.scala:303)
	at org.apache.spark.sql.catalyst.trees.CurrentOrigin$.withOrigin(TreeNode.scala:76)
	at org.apache.spark.sql.catalyst.trees.TreeNode.transformUp(TreeNode.scala:302)
	at org.apache.spark.sql.catalyst.trees.TreeNode$$anonfun$5.apply(TreeNode.scala:300)
	at org.apache.spark.sql.catalyst.trees.TreeNode$$anonfun$5.apply(TreeNode.scala:300)
	at org.apache.spark.sql.catalyst.trees.TreeNode$$anonfun$8.apply(TreeNode.scala:353)
	at org.apache.spark.sql.catalyst.trees.TreeNode.mapProductIterator(TreeNode.scala:207)
	at org.apache.spark.sql.catalyst.trees.TreeNode.mapChildren(TreeNode.scala:351)
	at org.apache.spark.sql.catalyst.trees.TreeNode.transformUp(TreeNode.scala:300)
	at org.apache.spark.sql.catalyst.plans.QueryPlan$$anonfun$transformExpressionsUp$1.apply(QueryPlan.scala:94)
	at org.apache.spark.sql.catalyst.plans.QueryPlan$$anonfun$transformExpressionsUp$1.apply(QueryPlan.scala:94)
	at org.apache.spark.sql.catalyst.plans.QueryPlan$$anonfun$3.apply(QueryPlan.scala:106)
	at org.apache.spark.sql.catalyst.plans.QueryPlan$$anonfun$3.apply(QueryPlan.scala:106)
	at org.apache.spark.sql.catalyst.trees.CurrentOrigin$.withOrigin(TreeNode.scala:76)
	at org.apache.spark.sql.catalyst.plans.QueryPlan.transformExpression$1(QueryPlan.scala:105)
	at org.apache.spark.sql.catalyst.plans.QueryPlan.org$apache$spark$sql$catalyst$plans$QueryPlan$$recursiveTransform$1(QueryPlan.scala:117)
	at org.apache.spark.sql.catalyst.plans.QueryPlan$$anonfun$org$apache$spark$sql$catalyst$plans$QueryPlan$$recursiveTransform$1$2.apply(QueryPlan.scala:122)
	at scala.collection.TraversableLike$$anonfun$map$1.apply(TraversableLike.scala:234)
	at scala.collection.TraversableLike$$anonfun$map$1.apply(TraversableLike.scala:234)
	at scala.collection.mutable.ResizableArray$class.foreach(ResizableArray.scala:59)
	at scala.collection.mutable.ArrayBuffer.foreach(ArrayBuffer.scala:48)
	at scala.collection.TraversableLike$class.map(TraversableLike.scala:234)
	at scala.collection.AbstractTraversable.map(Traversable.scala:104)
	at org.apache.spark.sql.catalyst.plans.QueryPlan.org$apache$spark$sql$catalyst$plans$QueryPlan$$recursiveTransform$1(QueryPlan.scala:122)
	at org.apache.spark.sql.catalyst.plans.QueryPlan$$anonfun$4.apply(QueryPlan.scala:127)
	at org.apache.spark.sql.catalyst.trees.TreeNode

In [6]:
df2=df1.withColumn('concat',F.split((F.concat_ws(',',(*(x for x in df.columns[1:])))),','))\
                 .drop(*(x for x in df.columns[1:])).groupBy().pivot(df.columns[0]).agg(F.first("concat"))

In [7]:
df2.withColumn("met", F.explode(F.arrays_zip(*(df2.columns)))).show(truncate=False)

+-----------------------------------------------+-----------------+---------------------+
id_1 |id_2 |met |
+-----------------------------------------------+-----------------+---------------------+
[int_key, string_key, double_key, 1, asd, null]|[double_key, 2.0]|[int_key, double_key]|
[int_key, string_key, double_key, 1, asd, null]|[double_key, 2.0]|[string_key, 2.0] |
[int_key, string_key, double_key, 1, asd, null]|[double_key, 2.0]|[double_key,] |
[int_key, string_key, double_key, 1, asd, null]|[double_key, 2.0]|[1,] |
[int_key, string_key, double_key, 1, asd, null]|[double_key, 2.0]|[asd,] |
[int_key, string_key, double_key, 1, asd, null]|[double_key, 2.0]|[null,] |
+-----------------------------------------------+-----------------+---------------------+

In [8]:
def transpose(df):
           df1= df.withColumn('concat',F.split((F.concat_ws(',',(*(x for x in df.columns[1:])))),','))\
                 .drop(*(x for x in df.columns[1:])).groupBy().pivot(df.columns[0]).agg(F.first("concat"))
           return df1.withColumn("met", F.explode(F.arrays_zip(*(df1.columns))))\
                 .select("met.*")
df.show()
transpose(df).show()

In [9]:
list=[[52,67,'boy','car'],
      [44,25,'girl','bike'],
      [98,85,'boy','car'],
      [52,41,'girl','car']]
df=spark.createDataFrame(list,['Column_A','Column_B','Column_C','Column_D'])
df.show()

+--------+--------+--------+--------+
Column_A|Column_B|Column_C|Column_D|
+--------+--------+--------+--------+
 52| 67| boy| car|
 44| 25| girl| bike|
 98| 85| boy| car|
 52| 41| girl| car|
+--------+--------+--------+--------+

In [10]:
from pyspark.sql.window import Window
w=Window().partitionBy("Column_C","Column_D").orderBy("Column_A")
df.withColumn("diff", F.sum(F.abs(F.col("Column_A")-F.col("Column_B"))).over(w)).show()

+--------+--------+--------+--------+----+
Column_A|Column_B|Column_C|Column_D|diff|
+--------+--------+--------+--------+----+
 52| 67| boy| car| 15|
 98| 85| boy| car| 28|
 44| 25| girl| bike| 19|
 52| 41| girl| car| 11|
+--------+--------+--------+--------+----+

In [11]:
from pyspark.sql import functions as F
from pyspark.sql.types import *

list=[[0,1,2,{'d': 1, 'e': 2}],
    [1,3,4,{'d': 7, 'e': 0}],
    [2,5,6,{'d': 5, 'e': 4}]]
df=spark.createDataFrame(list, ['id','a','b','dic'])
df.show()

+---+---+---+----------------+
 id| a| b| dic|
+---+---+---+----------------+
 0| 1| 2|[d -> 1, e -> 2]|
 1| 3| 4|[d -> 7, e -> 0]|
 2| 5| 6|[d -> 5, e -> 4]|
+---+---+---+----------------+

In [12]:
from pyspark.sql import functions as F
from pyspark.sql.types import *
schema=StructType([StructField("c", StringType()),
                  StructField("d", StringType())])
df.select("id","a","b", F.from_json("dic",schema)).show()

--------------------------------------------------------------------------- 
 Py4JJavaError Traceback (most recent call last)
 /databricks/spark/python/pyspark/sql/utils.py in deco (*a, **kw) 
 62 try : 
 ---> 63 return f ( * a , ** kw ) 
 64 except py4j . protocol . Py4JJavaError as e : 

 /databricks/spark/python/lib/py4j-0.10.7-src.zip/py4j/protocol.py in get_return_value (answer, gateway_client, target_id, name) 
 327 "An error occurred while calling {0}{1}{2}.\n" . 
 --> 328 format(target_id, ".", name), value)
 329 else : 

 Py4JJavaError : An error occurred while calling o1018.select.
: org.apache.spark.sql.AnalysisException: cannot resolve 'jsontostructs(`dic`)' due to data type mismatch: argument 1 requires string type, however, '`dic`' is of map<string,bigint> type.;;
'Project [id#87L, a#88L, b#89L, jsontostructs(StructField(c,StringType,true), StructField(d,StringType,true), dic#90, Some(Etc/UTC)) AS jsontostructs(dic)#108]
+- LogicalRDD [id#87L, a#88L, b#89L, dic#90], false

	at org.apache.spark.sql.catalyst.analysis.package$AnalysisErrorAt.failAnalysis(package.scala:42)
	at org.apache.spark.sql.catalyst.analysis.CheckAnalysis$$anonfun$checkAnalysis$1$$anonfun$apply$3.applyOrElse(CheckAnalysis.scala:126)
	at org.apache.spark.sql.catalyst.analysis.CheckAnalysis$$anonfun$checkAnalysis$1$$anonfun$apply$3.applyOrElse(CheckAnalysis.scala:111)
	at org.apache.spark.sql.catalyst.trees.TreeNode$$anonfun$6.apply(TreeNode.scala:303)
	at org.apache.spark.sql.catalyst.trees.TreeNode$$anonfun$6.apply(TreeNode.scala:303)
	at org.apache.spark.sql.catalyst.trees.CurrentOrigin$.withOrigin(TreeNode.scala:76)
	at org.apache.spark.sql.catalyst.trees.TreeNode.transformUp(TreeNode.scala:302)
	at org.apache.spark.sql.catalyst.trees.TreeNode$$anonfun$5.apply(TreeNode.scala:300)
	at org.apache.spark.sql.catalyst.trees.TreeNode$$anonfun$5.apply(TreeNode.scala:300)
	at org.apache.spark.sql.catalyst.trees.TreeNode$$anonfun$8.apply(TreeNode.scala:353)
	at org.apache.spark.sql.catalyst.trees.TreeNode.mapProductIterator(TreeNode.scala:207)
	at org.apache.spark.sql.catalyst.trees.TreeNode.mapChildren(TreeNode.scala:351)
	at org.apache.spark.sql.catalyst.trees.TreeNode.transformUp(TreeNode.scala:300)
	at org.apache.spark.sql.catalyst.plans.QueryPlan$$anonfun$transformExpressionsUp$1.apply(QueryPlan.scala:94)
	at org.apache.spark.sql.catalyst.plans.QueryPlan$$anonfun$transformExpressionsUp$1.apply(QueryPlan.scala:94)
	at org.apache.spark.sql.catalyst.plans.QueryPlan$$anonfun$3.apply(QueryPlan.scala:106)
	at org.apache.spark.sql.catalyst.plans.QueryPlan$$anonfun$3.apply(QueryPlan.scala:106)
	at org.apache.spark.sql.catalyst.trees.CurrentOrigin$.withOrigin(TreeNode.scala:76)
	at org.apache.spark.sql.catalyst.plans.QueryPlan.transformExpression$1(QueryPlan.scala:105)
	at org.apache.spark.sql.catalyst.plans.QueryPlan.org$apache$spark$sql$catalyst$plans$QueryPlan$$recursiveTransform$1(QueryPlan.scala:117)
	at org.apache.spark.sql.catalyst.plans.QueryPlan$$anonfun$org$apache$spark$sql$catalyst$plans$QueryPlan$$recursiveTransform$1$2.apply(QueryPlan.scala:122)
	at scala.collection.TraversableLike$$anonfun$map$1.apply(TraversableLike.scala:234)
	at scala.collection.TraversableLike$$anonfun$map$1.apply(TraversableLike.scala:234)
	at scala.collection.mutable.ResizableArray$class.foreach(ResizableArray.scala:59)
	at scala.collection.mutable.ArrayBuffer.foreach(ArrayBuffer.scala:48)
	at scala.collection.TraversableLike$class.map(TraversableLike.scala:234)
	at scala.collection.AbstractTraversable.map(Traversable.scala:104)
	at org.apache.spark.sql.catalyst.plans.QueryPlan.org$apache$spark$sql$catalyst$plans$QueryPlan$$recursiveTransform$1(QueryPlan.scala:122)
	at org.apache.spark.sql.catalyst.plans.QueryPlan$$anonfun$4.apply(QueryPlan.scala:127)
	at org.apache.spark.sql.catalyst.trees.TreeNode.mapProductIterator(TreeNode.scala:207)
	at org.apache.spark.sql.catalyst.plans.QueryPlan.mapExpressions(QueryPlan.scala:127)
	at org.apache.spark.sql.catalyst.plans.QueryP

In [13]:
list=[[20170924.00000]]
df=spark.createDataFrame(list,["date"])

df.show(truncate=False)

+-----------+
date |
+-----------+
2.0170924E7|
+-----------+

In [14]:
df.withColumn("date", F.to_date(F.col("date").cast("int").cast("string"), "yyyyMMdd")).show()

+----------+
 date|
+----------+
2017-09-24|
+----------+

In [15]:
from pyspark.sql import functions as F

In [16]:
root_path = './data/'
data=['1234.txt', '1235.txt']
[(root_path+x) for x in data]

Out[2]: ['./data/1234.txt', './data/1235.txt']

In [17]:
root_path = './data/'
file_list = os.listdir(root_path)

df= spark.read.text([(root_path+x) for x in file_list])

In [18]:
from pyspark.sql import functions as F
from pyspark.sql.functions import *
list=[['aa','aa'],
      ['bb','cc'],
      [None,None]]
df=spark.createDataFrame(list, ['COL_1','COL_2'])
df.show()

+-----+-----+
COL_1|COL_2|
+-----+-----+
 aa| aa|
 bb| cc|
 null| null|
+-----+-----+

In [19]:
df.withColumn("COL_3", when(col("COL_1")==col("COL_2"), lit("Valid")).when((col("COL_1")==None) & (col("COL_2")==None),lit(None)).when(col("COL_1")!=col("COL_2"),lit("Invalid"))).show()

+-----+-----+-------+
COL_1|COL_2| COL_3|
+-----+-----+-------+
 aa| aa| Valid|
 bb| cc|Invlaid|
 null| null| null|
+-----+-----+-------+

In [20]:
df.withColumn("COL_3",when(col('col_1') == col('col_2'), lit('valid')).when(col('col_1') != col('col_2'), lit('invalid'))).show()

+-----+-----+-------+
COL_1|COL_2| COL_3|
+-----+-----+-------+
 aa| aa| valid|
 bb| cc|invalid|
 null| null| null|
+-----+-----+-------+

In [21]:
df.withColumn('COL_3', when((col("COL_1") == col("COL_2")), 'Valid').when((col("COL_1") != col("COL_2")), 'Invalid')).show()

+-----+-----+-------+
COL_1|COL_2| COL_3|
+-----+-----+-------+
 aa| aa| Valid|
 bb| cc|Invalid|
 null| null| null|
+-----+-----+-------+

In [22]:
df.withColumn('COL_3', when((col("COL_1") == col("COL_2")), 'Valid').when((col("COL_1") != col("COL_2")), 'Invalid').otherwise(lit(None))).show()

+-----+-----+-------+
COL_1|COL_2| COL_3|
+-----+-----+-------+
 aa| aa| Valid|
 bb| cc|Invalid|
 null| null| null|
+-----+-----+-------+

In [23]:
df.groupBy("COL_1").agg(F.first("COL_2")).show()

+-----+-------------------+
COL_1|first(COL_2, false)|
+-----+-------------------+
 null| null|
 bb| cc|
 aa| aa|
+-----+-------------------+

In [24]:
from pyspark.sql import functions as F
from pyspark.sql.types import *

list=[[1,0,1,0,2,'tre',100],
     [2,1,0,0,0,'tre',200],
     [3,2,0,1,1,'abc',300]]

df=spark.createDataFrame(list, ['id','pt1','pt2','pt3','pt5','name','amt'])
df.show()
df.withColumn("pt", F.greatest(*[x for x in df.columns if x.startswith('pt')])).show()

+---+---+---+---+---+----+---+
 id|pt1|pt2|pt3|pt5|name|amt|
+---+---+---+---+---+----+---+
 1| 0| 1| 0| 2| tre|100|
 2| 1| 0| 0| 0| tre|200|
 3| 2| 0| 1| 1| abc|300|
+---+---+---+---+---+----+---+

In [25]:
df.withColumn("pt", F.greatest(*[x for x in df.columns if x.startswith('pt')])).show()

+---+---+---+---+---+----+---+---+
 id|pt1|pt2|pt3|pt5|name|amt| pt|
+---+---+---+---+---+----+---+---+
 1| 0| 1| 0| 2| tre|100| 2|
 2| 1| 0| 0| 0| tre|200| 1|
 3| 2| 0| 1| 1| abc|300| 2|
+---+---+---+---+---+----+---+---+

In [26]:
list=[['red'],
     ['yellow'],
     ['green'],
     ['yo']]
df= spark.createDataFrame(list, ['CODE'])
df.show()

+------+
 CODE|
+------+
 red|
yellow|
 green|
 yo|
+------+

In [27]:
from pyspark.sql.functions import when,col,lit

df.withColumn("TYPE", when((col("CODE")=="red")| (col("CODE")=="yellow"),lit("X")).when(col("CODE")=="green",lit("Y")).otherwise(lit("N"))).show()

+------+----+
 CODE|TYPE|
+------+----+
 red| X|
yellow| X|
 green| Y|
 yo| N|
+------+----+

In [28]:
df = df.withColumn("TYPE",
((Column) when(df.col("CODE").$eq$bang$eq("asdfgh"),(functions.lit("X")))